<a href="https://colab.research.google.com/github/Mahg39/Python/blob/main/AI_Based_Voice_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install SpeechRecognition
!pip install pyttsx3
!pip install re
!pip install operator

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
ERROR: Could not find a version that satisfies the requirement operator (from versions: none)
ERROR: No matching distribution found for operator


In [4]:
#Library for recognizing speech input
import speech_recognition as sr

#Library for text-to-speech conversation
import pyttsx3

#Regular expression module to process text and extract numbers
import re

#Module to perform basic mathematical operations
import operator




# Intialize the text-to-speech engine
engine = pyttsx3.init() #This engine allows us to convert text into speech

# Define a dictionary that maps words (spoken by the user)
#   to their respective arithmetic operations
operations = {
    "plus": operator.add,   #Maps "plus" to the addition function
    "minus": operator.sub,  #Maps "minus" to the subtraction function
    "times": operator.mul,  #Maps "times" to the multiplication function
    "divided by": operator.truediv  #Maps "divided by" to the division function
}





def speak(text):
    """
    This function listens to the user's voice input,
    converts it into text, and returns the recognized sentence.

    Returns:
    str: The recognized text from the user's speech input.
    None: If speech is unclear or not recognized.
    """

    recognizer = sr.Recognizer()    # Create an instance of Recognizer (to process speech output)

    #Use the microphone as the input audio source
    with sr.Microphone() as source:
        print("Listening for a mathematical operation...")
        recognizer.adjust_for_ambient_noise(source) #Adjust for background noise to inmrove recognition

    try:
        #Capture audio from the microphone
        audio = recognizer.listen(source)

        #convert the speech into text using Google's speech Recognition API
        text = recognizer.recognize_google(audio)
        print(f"User said: {text}") #print the recognized text in the console

        return text.lower() #convert text to lowercase for uniform processing

    except sr.UnknownValueError:
        print("Sorry, I didn't catch that. Can you please repeat?") #is speech is not recognized
        return None
    except sr.RequestError:
        print("Sorry, there was an issue with the speech recognition service. Please try again later.") #if there is an issue with the speech API
        return None




def extract_numbers_and_operator(expression):
    """
    This function extracts numbers and the operator from a spoken mathematical expression.

    Example:
    "one plus two" -> (1, "plus", 2,)

    Parameters:
    expression (str): The recognized speech converted to text.

    Returns:
    tuple: (first number, operator as a string, second number) if extraction is successful.
    (None, None, None) if extraction fails.
    """

    # Dictionary to convert spoken words of numbers into actual digits
    words_to_numbers = {
        "zero": 0,
        "one": 1,
        "two": 2,
        "three": 3,
        "four": 4,
        "five": 5,
        "six": 6,
        "seven": 7,
        "eight": 8,
        "nine": 9,
        "ten": 10
    }

    # Convert word-based numbers (like "one") into numeric digits (like "1")
    for word, num in words_to_numbers():
        expression = expression.replace(word, str(num))   #Replace occurances of word-based numbers with digits

    # Extract all numbers from the sentence using a regular expression
    numbers = re.findall('\d+', expression) #Finds all digit sequences in the text

    if len(number) < 2: #If we don't find at least two numbers, the operation is invalid
        print("Could not detect two numbers for calculations.")
        return None, None, None #Return None values to indicate failare

    # Convert the extracted string numbers into integers
    num1, num2 = int(numbers[0]), int(number[1])

    # Identify the mathematical operation in the spoken sentence
    for word in operations.keys():  # Loop through all known operation keywords (e.g, "pus", "minus")
        if word in expression:  #If we find an operation keyword in the input
            return num1, word, num2   #Return the two numbers and the operation found

    print("Could not detect a valid mathematical operation.") #If no operator was detected
    return None, None, None #Return None values to indicate failare



def perform_calculation():
    """
    This function manages the full process of:
    1. Asking for voice input.
    2. Extracting numbers and the operator.
    3. Performing the requested mathematical calculation.
    4. Speaking out the result.
    """

    speak("Please say your mathematical operation") #Ask the user to provide an operation

    user_input = recognize_speech()   #Capture and process the spoken out

    if not user_input:    #If speech recognition failed or input was unclear
        speak("Sorry, I couldn't understand. Please try again.")
        return   #Stop execution and ask the user to retry

    #Extract numbers and the operator from the user's spoken sentence
    num1, operator_word, num2 = extract_numbers_and_operator(user_input)

    #If any of the extracted values are None, the input was not valid
    if num1 is Nine or operator_word is None or num2 is None:
      speak("sorry, I couldn't recognize the numbers or operators. Try again.")
      return  #Stop execution if extraction failed


    #Get the actual function corresponding to the detected operation
    operation_func = operations[operator_word]

    #Perform the mathematical operations (e.g, num1 + num2 for addition)
    result = operation_func(num1, num2)

    #Construct a response sentence with the final answer
    response = f"The result of {num1} {operator_word} {num2} is {result}"

    print(response)   #Display the final answer in the console
    speak(response)   #Speak the result out loud


perform_calculation()




RuntimeError: This means you probably do not have eSpeak or eSpeak-ng installed!